In [143]:
import nltk
import itertools
import string
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
_pronunciations = nltk.corpus.cmudict.dict()

In [37]:
_pronunciations["he'd"]

[[u'HH', u'IY1', u'D']]

In [53]:
def num_syllables(word):
    if word not in _pronunciations:
            return 1
    else:
        pronuns = _pronunciations[word]

        shortest_syllables = -1

        for this_pronun in pronuns:
            this_num_syllables = len([s for s in this_pronun if s[-1].isdigit()])
            if shortest_syllables == -1:
                shortest_syllables = this_num_syllables
            elif this_num_syllables < shortest_syllables:
                shortest_syllables = this_num_syllables
        if shortest_syllables == 0:
            return 1
        return shortest_syllables

In [131]:
num_syllables("forgot")

2

In [55]:
def rhymes(a, b):
    # get number of syllables for words a, b
    a_num_syllables = num_syllables(a)
    b_num_syllables = num_syllables(b)

    # check which word has more syllables
    if a_num_syllables < b_num_syllables:
        shorter = a
        longer = b
    else:
        shorter = b
        longer = a
    
    
    if shorter not in _pronunciations or longer not in _pronunciations:
        return False
    else:
        shorter_pronuns = _pronunciations[shorter]
        longer_pronuns = _pronunciations[longer]
        for s_pronun in shorter_pronuns:
            try:
                shorter_sounds = s_pronun[[1 if s[-1].isdigit() else 0 for s in s_pronun].index(1):]
            except ValueError:
                # If a word's pronounciation doesn't have any vowels, it rhymes with everything
                return True
            last_n = 0-len(shorter_sounds)
            for l_pronun in longer_pronuns:
                longer_sounds = l_pronun[[1 if s[-1].isdigit() else 0 for s in l_pronun].index(1):]
                longer_sounds_last_n = longer_sounds[last_n:]
                if shorter_sounds == longer_sounds_last_n:
                    return True


    return False

In [132]:
rhymes("forgot", "trot")

True

In [57]:
rhymes("plane", "train")

True

In [201]:
def is_limerick(text):
    punctuation_to_avoid = [s for s in punctuation]
    lines = text.strip().split('\n')
    # There must be 5 lines in a limerick AABBA
    if len(lines) != 5:
        return False

    # contains number of syllables for corresponding line index
    num_sylls = []

    # contains the last word of each line
    last_word = []

    for line in lines:
        syllables = 0
#         line_words = line.strip().split(" ")
        line_words = word_tokenize(line.strip())
        line_words = [w for w in line_words if w not in punctuation_to_avoid]
        for line_word in line_words:
            syllables += num_syllables(line_word)
        print line,syllables
        if syllables < 4:
            print "sdfdsf"
            return False
        num_sylls.append(syllables)
        last_word.append(line_words[-1].translate(None, string.punctuation))

    A_lines_last_word = [last_word[0],last_word[1],last_word[4]]
    B_lines_last_word = [last_word[2],last_word[3]]

    A_syllables = [num_sylls[0],num_sylls[1],num_sylls[4]]
    B_syllables = [num_sylls[2],num_sylls[3]]

    # make sure A lines rhyme with each other
    if any(not rhymes(x.lower(),y.lower()) for x,y in itertools.combinations(A_lines_last_word,2)):
        return False

    # make sure B lines rhyme with each other
    if not rhymes(B_lines_last_word[0].lower(),B_lines_last_word[1].lower()):
        return False

    # make sure A lines don't rhyme with B lines
    if any(rhymes(x.lower(),y.lower()) for x,y in itertools.product(A_lines_last_word,B_lines_last_word)):
        return False

    # No two A lines should differ in their number of syllables by more than two
    if any(abs(x-y) > 2 for x,y in itertools.combinations(A_syllables,2)):
        return False

    # The B lines should differ in their number of syllables by no more than two
    if abs(B_syllables[0]-B_syllables[1]) > 2:
        return False

    # Each of the B lines should have fewer syllables than each of the A lines
    if any(x-y <= 0 for x,y in itertools.product(A_syllables,B_syllables)):
        return False

    return True

In [217]:
g = """Computers have become so intelligent and smart
Google artificial intelligence can draw art
Watson won jeopardy, earning fame
To defeat cancer is now his aim
The future of AI looks awesome, this is just the start."""
is_limerick(g)

Computers have become so intelligent and smart 11
Google artificial intelligence can draw art 12
Watson won jeopardy, earning fame 8
To defeat cancer is now his aim 9
The future of AI looks awesome, this is just the start. 13


True

In [189]:
g = """There was a young lady one fall
Who wore a newspaper dress to a ball.
The dress caught fire
And burned her entire
Front page, sporting section and all."""

In [190]:
is_limerick(g)

['There', 'was', 'a', 'young', 'lady', 'one', 'fall']
['Who', 'wore', 'a', 'newspaper', 'dress', 'to', 'a', 'ball']
['The', 'dress', 'caught', 'fire']
['And', 'burned', 'her', 'entire']
['Front', 'page', 'sporting', 'section', 'and', 'all']


True

In [180]:
print punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [17]:
a.strip().split('\n')

['a woman whose friends called a prude',
 'on a lark when bathing all nude',
 'saw a man come along',
 'and unless we are wrong',
 'you expected this line to be lewd']

In [16]:
a.strip()

'a woman whose friends called a prude\non a lark when bathing all nude\nsaw a man come along\nand unless we are wrong\nyou expected this line to be lewd'

In [69]:
is_limerick(a)

True

In [72]:
"aSFDSF".lower()

'asfdsf'

In [73]:
num_syllables("he'd")

1

In [65]:
word_tokenize("he can't work!s")

['he', 'ca', "n't", 'work', '!', 's']

In [114]:
def apostrophe_tokenize(line):
    return re.findall(r"[\w']+|[!#$%&\"()*+,-./:;<=>?@[\]^_`{|}~]", line)

In [115]:
apostrophe_tokenize("he can't work!sdfdsf ")

['he', "can't", 'work', '!', 'sdfdsf']

In [127]:
def guess_syllables(word):
    vowels = ['a','e','i','o','u']
    word = word.lower()
    
    # count the number of vowels in the word
    count = 0
    for w in word:
        if w in vowels:
            count += 1
    # subtract silent vowels
    # any words that end with 'e' or consectutive vowels
    if word[-1] == 'e':
        count -= 1
    previous = False
    for w in word:
        if w in vowels:
            if previous:
                count -= 1
            else:
                previous = True
        else:
            previous = False
        
        
    return count

In [130]:
guess_syllables('Asia')

2